In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 35495664
paper_name = 'cao_liu_2022' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
sheet_names = ['First screening','Second screening','Third screening']

In [6]:
original_data_all = []
for sheet_name in sheet_names:
    original_data = pd.read_excel('raw_data/Table_1_Genome-Wide Identification of Cellular Pathways and Key Genes That Respond to Sodium Bicarbonate Stress in Saccharomyces cerevisiae.XLSX', 
                                  sheet_name=sheet_name)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    original_data['orf'] = original_data['Array ORF'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    original_data['data'] = original_data['Score'].astype(float)
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    
    original_data_all.append(original_data)

Original data dimensions: 4129 x 12
  Query ORF Query Name Array ORF Array Name  Array annotation  \
0    NAHCO3     NAHCO3   YOR202W       HIS3               NaN   
1    NAHCO3     NAHCO3   YBL032W       HEK2               NaN   
2    NAHCO3     NAHCO3   YBL031W       SHE1               NaN   
3    NAHCO3     NAHCO3   YBL029W    YBL029W               NaN   
4    NAHCO3     NAHCO3   YBL028C    YBL028C               NaN   

   Normalized colony size (EXPERIMENT)  \
0                              0.98791   
1                              1.20614   
2                              0.95499   
3                              1.26784   
4                              1.47208   

   Normalized colony std. dev. (EXPERIMENT)  Normalized colony size (CONTROL)  \
0                                     0.178                           1.04016   
1                                     0.078                           1.00875   
2                                     0.036                           0.89638

In [7]:
original_data = pd.concat(original_data_all, axis=1)

In [8]:
original_data.shape

(4081, 3)

In [10]:
original_data['data_mean'] = original_data.mean(axis=1)

In [12]:
original_data = original_data[['data_mean']].copy()
original_data.columns = ['data']

# Prepare the final dataset

In [13]:
data = original_data.copy()

In [14]:
dataset_ids = [22228]
datasets = datasets.reindex(index=dataset_ids)

In [15]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [16]:
data.head()

dataset_id,22228
data_type,value
orf,
YAL002W,-0.230927
YAL004W,-0.216047
YAL005C,-0.286283
YAL007C,-0.174313
YAL008W,0.184797


## Subset to the genes currently in SGD

In [17]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 20


In [18]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22228
,data_type,value
gene_id,orf,
2,YAL002W,-0.230927
1863,YAL004W,-0.216047
4,YAL005C,-0.286283
5,YAL007C,-0.174313
6,YAL008W,0.184797


# Normalize

In [19]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [20]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [21]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          22228          
data_type           value    valuez
gene_id orf                        
2       YAL002W -0.230927 -1.693574
1863    YAL004W -0.216047 -1.610021
4       YAL005C -0.286283 -2.004406
5       YAL007C -0.174313 -1.375685
6       YAL008W  0.184797  0.640750

# Print out

In [22]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [23]:
from IO.save_data_to_db3 import *

In [24]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 35495664...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.07s/it]

Updating the data_modified_on field...
